- Tactical assset allcoation 이란? 
- 변동성 낮은 것 + 3~12개월 모멘텀 zscore 상위 n개 선출 하여 매월 말 리밸런싱

In [1]:
import pandas as pd
from pandas import Series, DataFrame
from pandas_datareader import data
from pandas.tseries.offsets import Day, MonthEnd
import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy.stats import rankdata

In [5]:
##수정 주가 사용
file = pd.ExcelFile(r'C:\Users\user\혜성\금융공학_bizlab\Quant trading\MPAA\MPAA 데이터.xlsx')
assets = file.sheet_names
data = {}
for asset in assets:
    data[asset]= file.parse(asset)
    data[asset].index = data[asset].날짜
    data[asset].drop('날짜',axis=1,inplace = True)

In [15]:
data.keys()

dict_keys(['국가', '섹터', '팩터', '채권', '현금'])

In [24]:
fee = 0.0030
lookback = 12
num = 5

In [23]:
prices = data['국가']
s = pd.Series(np.arange(prices.shape[0]), index=prices.index)
ep = s.resample("M", how="max")
ep

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).max()
  This is separate from the ipykernel package so we can avoid doing imports until


12

In [28]:
cumret = prices.iloc[ep[12]] / prices.iloc[ep[12-12]] - 1
rankdata(-cumret)

array([ 5.,  4., 10., 12.,  2.,  3., 11.,  6.,  9.,  8.,  1.,  7.])

In [25]:
wts = list()

for i in range(lookback, len(ep)) :
    ## prices.index[ep[i]]       check the calendar
    cumret = prices.iloc[ep[i]] / prices.iloc[ep[i-12]] - 1
    K = rankdata(-cumret) <= num
    
    wt = np.repeat(0.00, prices.shape[1], axis = 0)
    wt[K] = 1 / num
    wt = pd.DataFrame(data = wt.reshape(1,prices.shape[1]),
                      index = [prices.index[ep[i]]],
                      columns = [prices.columns])
    wts.append(wt)
    
wts = pd.concat(wts)

In [42]:
pd.DataFrame(np.where(prices.pct_change(12).dropna().rank(1,ascending =0)<6,0.2,0),columns=prices.columns)

,S&P 500,러셀 3000,니케이 225,항셍,홍콩 H,대만 가권,상해종합,영국 FTSE 100,프랑스 CAC 40,독일 DAX,KOSPI,인도 SENSEX
0,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.0
1,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.0
2,0.2,0.2,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.0
3,0.0,0.2,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2
4,0.0,0.0,0.2,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.2,0.0
5,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.2,0.0,0.0,0.2,0.2
6,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0.0,0.2,0.2
7,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0.0,0.2,0.2
8,0.0,0.0,0.2,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2
9,0.0,0.0,0.0,0.2,0.2,0.2,0.0,0.0,0.0,0.0,0.2,0.2


In [45]:
 np.repeat(0.00, prices.shape[1], axis = 0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])